In [1]:
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout
from keras.utils import to_categorical
import os

In [2]:
# Load MIDI files and extract notes
def get_notes():
    notes = []
    for file in os.listdir("midi_files"):
        if file.endswith(".mid"):
            midi = converter.parse(os.path.join("midi_files", file))
            notes_to_parse = None
            
            # Get all notes and chords from the MIDI file
            try:
                notes_to_parse = midi.flat.notes
            except Exception as e:
                print(f"Error parsing {file}: {e}")
                continue
            
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
    
    return notes

In [3]:
# Prepare sequences for training
def prepare_sequences(notes, sequence_length=100):
    pitch_names = sorted(set(notes))
    note_to_int = {note: number for number, note in enumerate(pitch_names)}
    
    network_input = []
    network_output = []
    
    for i in range(len(notes) - sequence_length):
        seq_in = notes[i:i + sequence_length]
        seq_out = notes[i + sequence_length]
        network_input.append([note_to_int[n] for n in seq_in])
        network_output.append(note_to_int[seq_out])
    
    n_patterns = len(network_input)
    n_vocab = len(pitch_names)
    
    # Reshape input into a format compatible with LSTM
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)  # Normalize input
    network_output = to_categorical(network_output, num_classes=n_vocab)
    
    return network_input, network_output


In [4]:
# Build the LSTM model
def create_model(input_shape, n_vocab):
    model = Sequential()
    model.add(LSTM(256, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [9]:
def generate_music(model, network_input, int_to_note, n_vocab, sequence_length=500):
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start]
    
    prediction_output = []
    for note_index in range(sequence_length):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        # Update the pattern
        index_array = np.array([[index]])  # (1, 1)
        index_array = np.reshape(index_array, (1, 1))  # Reshape to match the pattern shape
        
        pattern = np.append(pattern, index_array, axis=0)  # Append along the correct axis
        pattern = np.delete(pattern, 0, axis=0)  # Remove the first element

    return prediction_output



In [11]:
if __name__ == "__main__":
    notes = get_notes()
    network_input, network_output = prepare_sequences(notes)
    
    model = create_model((network_input.shape[1], network_input.shape[2]), len(set(notes)))
    model.fit(network_input, network_output, epochs=100, batch_size=64)

    # Save the model
    model.save("music_generator.keras")

    # Generate music
    int_to_note = {number: note for number, note in enumerate(sorted(set(notes)))}
    generated_notes = generate_music(model, network_input, int_to_note, len(set(notes)))

    # Save generated music as MIDI
    offset = 0
    output_notes = []

    for item in generated_notes:
        if ('.' in item) or item.isdigit():  # If it's a chord
            notes_in_chord = item.split('.')
            notes = [note.Note(int(n)) for n in notes_in_chord]
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:  # If it's a note
            new_note = note.Note(item)
            new_note.offset = offset
            output_notes.append(new_note)
        
        offset += 0.5  # Adjust the offset for the next note

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='generated_music.mid')

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 629ms/step - loss: 4.0498
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 518ms/step - loss: 3.6690
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 494ms/step - loss: 3.5263
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 488ms/step - loss: 3.5453
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 495ms/step - loss: 3.5431
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 511ms/step - loss: 3.5515
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 486ms/step - loss: 3.4901
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 523ms/step - loss: 3.5825
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 489ms/step - loss: 3.5601
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 475ms/step - loss: 3.5526
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 529ms/step - loss: 3.5289
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 489ms/step - loss: 3.6066
Epoch 13/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 487ms/step - loss: 3.5247
Epoch 14/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 489ms/step - loss: 3.5804
Epoch 15/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 451ms/step - loss: 3.5336
Epoc